# Full Implementation Notebook

This notebook contain the
complete working code where the Hugging Face text-to-text pipeline is
integrated with Gradio.

## Code

### Libraries

Start by installing and importing necessary libraries

In [ ]:
!pip install gradio
!pip install transformers

In [ ]:
import gradio as gr # Gradio library to create an interactive interface
from transformers import pipeline # Transformers libraries which imports pipeline to use Hugging-Face models
import pandas as pd # Pandas library for data manipulation and analysis
import matplotlib.pylab as plt # Matplot library for the interactive visualizations

### Initialize Sentiment Analyzers and Define Function

In this project, Two pre-trained sentiment analysis models will be used, One is for analysizng English text, and the other one if for Arabic text, Also the function that will be used by the Gradio


In [ ]:
# Initialize the analyzers

# Loads a pretrained model for the Arabic language
arabic_analyzer = pipeline('sentiment-analysis', model='CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment')

# Loads a pretrained model for the English language
english_analyzer = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

In [62]:
# Define function

def sentiment_analysis(language,file):
  # Select the appropriate analyzer
  if language == "Arabic":
    analyzer = arabic_analyzer
  else:
    analyzer = english_analyzer

  results = []

  # Read the sentences from the uploaded file
  with open (file.name,'r') as fn:
    sentences = fn.readlines()

  # Perform sentiment analysis on each sentence
  for sentence in sentences:
    result = analyzer(sentence)
    result = result[0]

    results.append({
        "Sentence": sentence,
        "Label": result['label'],
        "Score": f"{result['score'] *100:.2f}%"
    })
  # Convert the results into a DataFrame
  df = pd.DataFrame(results)
  # Ensure every label is lower, if not this will cause a logic error
  # English labels are Upper but Arabic labels are lower
  df['Label'] = df['Label'].str.lower()
  # Take the "Label" column values
  label_value = df['Label'].value_counts()

  # Pre-Define the plot parameters
  labels = ['Positive','Neutral','Negative']
  counts = [label_value.get('positive',0),
           label_value.get('neutral',0),
           label_value.get('negative',0)]
  colors=['green','gray','red']

  # Create a bar plot
  plt.bar(labels,counts,color=colors)
  plt.title('Sentiment-Analysis')
  plt.xlabel("Labels")
  plt.ylabel("No. of sentences")


  return df,plt

### Build The Gradio Interface

Here, The Gradio interface is set up using the following components:

**A file uploader:** Allows user to upload a text file which contains the sentences to be analyzed

**Dropdown Menu**: Allows user to choose in which language the sentences will be analyzed

**DataFrame Output:** Displays program's output in a structured table format

**Title and Description:** Provides clear title and description for the interface

In [65]:
# Set up the Gradio interface

demo = gr.Interface(
    fn=sentiment_analysis,
    inputs=[gr.Dropdown(choices=["Arabic","English"],
            label="Select a Language",
            value="Arabic"),
            gr.File(label="Upload a file")],
    outputs=[gr.DataFrame(label="Results"),
             gr.Plot(label="Bar plot")],
    title="Sentiment-Analysis",
    description="Gradio interface that allows users to Choose what language the sentences will be and upload a text file containing the sentences to be analyzed, the sentences will be classified and results will be in a formatted table along with a plot sentiment distribution"
)

demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://487a5d21908977ea69.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://487a5d21908977ea69.gradio.live
